In [1]:
import requests
import pprint
import pickle
login = {
    'username': 'dimensionsAPI@URMC.Rochester.edu',
    'password': 'Rochester&/()API'
}
resp = requests.post('https://app.dimensions.ai/api/auth.json', json=login)
resp.raise_for_status()
headers = {
    'Authorization': "JWT " + resp.json()['token']
}

###### Categories for FOR 

In [3]:
forbig=[]
for i in range(1,23):
    if i<10:
        forbig.append(" 0"+str(i)+" ")
    else:
        forbig.append(" "+str(i)+" ")
        
forsmall={}
forsmall["01"]=["01","02","03","04","05","99"]
forsmall["02"]=["01","02","03","04","05","06","99"]
forsmall["03"]=["01","02","03","04","05","06","07","99"]
forsmall["04"]=["01","02","03","04","05","06","99"]
forsmall["05"]=["01","02","03","99"]
forsmall["06"]=["01","02","03","04","05","06","07","08","99"]
forsmall["07"]=["01","02","03","04","05","06","07","99"]
forsmall["08"]=["01","02","03","04","05","06","07","99"]
forsmall["09"]=["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","99"]
forsmall["10"]=["01","02","03","04","05","06","07","99"]
forsmall["11"]=["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","99"]
forsmall["12"]=["01","02","03","04","05","99"]
forsmall["13"]=["01","02","03","99"]
forsmall["14"]=["01","02","03","99"]
forsmall["15"]=["01","02","03","04","05","06","07","99"]
forsmall["16"]=["01","02","03","04","05","06","07","08","99"]
forsmall["17"]=["01","02","99"]
forsmall["18"]=["01","02","99"]
forsmall["19"]=["01","02","03","04","05","99"]
forsmall["20"]=["01","02","03","04","05","99"]
forsmall["21"]=["01","02","03","99"]
forsmall["22"]=["01","02","03","04","99"]
for i in forsmall.keys():
    forsmall[i]=[" "+i+x+" " for x in forsmall[i]]   
    forsmall[i]=[f.strip(' ') for f in forsmall[i]]
    
with open('forbig','wb') as fp:
    pickle.dump(forbig,fp)
with open ('forsmall', 'wb') as fp:
    pickle.dump(forsmall,fp)

In [5]:
FORtext=[]
with open("FORtext.rtf","r") as file:
    for line in file:
        FORtext.append(line)
FORtext=FORtext[7:]
FORtext[0]=FORtext[0][FORtext[0].find("D"):]
for i in range(0,22):
    FORtext[i]=FORtext[i][FORtext[i].find(" ")+1:]
for i,s in enumerate(FORtext):
    FORtext[i] = FORtext[i].strip()
    FORtext[i] = FORtext[i][:-1]
menu={}
for i in FORtext:
    menu[i[:i.find(" ")]]=i[i.find(" "):]
del menu['\\outl0\\strokewidth0']
menu['101']="Pure Mathemetics"
for i in menu.keys():
    if len(i)==1:
        menu['0'+i] = menu.pop(i)
menu["01"]=menu["01"].rstrip()
menu["02"]=menu["02"].rstrip()

with open ('menu', 'wb') as fp:
    pickle.dump(menu,fp)

###### Querying of counts for FORs( from 01 to 22)

In [6]:
import pandas as pd
counts1=[]
for f in forbig:
    resp = requests.post(
        'https://app.dimensions.ai/api/dsl.json',
        data='search publications where FOR_first.name~"'+f+'" return publications[FOR_first]',
        headers=headers)
    j=resp.json()
    counts1.append(j['_stats']['total_count'])
forfirst=forbig
for_first=list(menu.values())[:22]

In [7]:
t1={"FORfirst":forfirst,"Name":for_first,"Counts":counts1}
d1 = pd.DataFrame(t1) 
d1.to_csv("FOR_first_counts.csv")

##### Counts for FORs (01-22)

In [16]:
d1 = pd.read_csv("FOR_first_counts.csv") 
d1

,Unnamed: 0,FORfirst,Name,Counts
0,0,1,TECHNOLOGY,2050765
1,1,2,MEDICAL AND HEALTH SCIENCES,2310856
2,2,3,BUILT ENVIRONMENT AND DESIGN,4478669
3,3,4,EDUCATION,1062380
4,4,5,ECONOMICS,748378
5,5,6,"COMMERCE, MANAGEMENT, TOURISM AND SERVICES",6152625
6,6,7,STUDIES IN HUMAN SOCIETY,351783
7,7,8,PSYCHOLOGY AND COGNITIVE SCIENCES,3598505
8,8,9,LAW AND LEGAL STUDIES,5871028
9,9,10,STUDIES IN CREATIVE ARTS AND WRITING,935939


###### Querying for counts for sub FORs 

In [126]:
forsecond=[]
for_second=[]
counts=[]
for i in forsmall.keys():
    counter=0
    for f in forsmall[i]:
        print(menu[str(int(f))])
        resp = requests.post(
            'https://app.dimensions.ai/api/dsl.json',
            data='search publications where FOR.name~"'+f+'" return publications[FOR]',
            headers=headers)
        j=resp.json()
        if j['publications']:
            print("paper number in [",j['publications'][0]['FOR'][0]['name'],"] is [", j['_stats']['total_count'],"]")
        else:
            print("paper number in [",f,"] is [", j['_stats']['total_count'],"]")
        counter+=j['_stats']['total_count']
        forsecond.append(f)
        for_second.append(menu[str(int(f))])
        counts.append(j['_stats']['total_count'])
    forsecond.append(i)
    for_second.append(menu[str(int(i))])
    counts.append(counter)
    print("-------------------------------that's it for FOR [",i,"]------------------------------------------")
    print("------------------------total number of FOR [",i,"] is",counter,"---------------------------------")


Pure Mathemetics
paper number in [ 0101 Pure Mathematics ] is [ 715735 ]
 Applied Mathematics
paper number in [ 0102 Applied Mathematics ] is [ 459426 ]
 Numerical and Computational Mathematics
paper number in [ 0103 Numerical and Computational Mathematics ] is [ 317074 ]
 Statistics
paper number in [ 0104 Statistics ] is [ 582641 ]
 Mathematical Physics
paper number in [ 0105 Mathematical Physics ] is [ 14962 ]
 Other Mathematical Sciences 


KeyboardInterrupt: 

In [99]:
for index,i in enumerate(forsmall.keys()):
    inst=forsecond.index(i)+1
    forsecond.insert(inst,i)
    for_second.insert(inst,"direct query")
    counts.insert(forsecond.index(i),counts1[index])
# incorporating two tables and compare numbers 

In [104]:
t={"FOR":forsecond,"Name":for_second,"Counts":counts}
d=pd.DataFrame(t)
d.to_csv("FORcounts.csv")

##### Counts for sub FORs

In [128]:
d = pd.read_csv("FORcounts.csv") 
d

,Unnamed: 0,FOR,Name,Counts
0,0,101,Pure Mathemetics,716899
1,1,102,Applied Mathematics,459426
2,2,103,Numerical and Computational Mathematics,316631
3,3,104,Statistics,583303
4,4,105,Mathematical Physics,14962
5,5,199,Other Mathematical Sciences,0
6,6,1,MATHEMATICAL SCIENCES,2034543
7,7,1,direct query,2091221
8,8,201,Astronomical and Space Sciences,206834
9,9,202,"Atomic, Molecular, Nuclear, Particle and Plas...",689770
